In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import hashlib
import json
import polars as pl

sys.path.append('.')
sys.path.append('..')



In [2]:
# df  = pd.read_csv('../data/resume_job_matching_dataset.csv')
# df["match_score"] = df["match_score"] *2
# df["data_id"] = "id_" + df.index.astype(str)
# df
# df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# df_train.to_csv('../data/train.csv', index=False)
# df_test.to_csv('../data/test.csv', index=False)

In [3]:
# import json
# outputs = []
# paths = [
#     '../data/output.json',
#     '../data/output2.json',
# ]
# for path in paths:
#     outputs.extend(
#         json.load(open(path, "r"))
#     )
# print(
#     *outputs, sep="\n"
# )

# Evaluation

In [4]:
df_train = pl.read_csv('../data/test.csv')
df_train

job_description,resume,match_score,data_id
str,str,i64,str
"""Product Manager needed with ex…","""Experienced professional skill…",8,"""id_6252"""
"""Product Manager needed with ex…","""Experienced professional skill…",4,"""id_4684"""
"""Data Scientist needed with exp…","""Experienced professional skill…",4,"""id_1731"""
"""Data Analyst needed with exper…","""Experienced professional skill…",4,"""id_4742"""
"""ML Engineer needed with experi…","""Experienced professional skill…",8,"""id_4521"""
…,…,…,…
"""Data Analyst needed with exper…","""Experienced professional skill…",6,"""id_8014"""
"""Product Manager needed with ex…","""Experienced professional skill…",6,"""id_1074"""
"""Product Manager needed with ex…","""Experienced professional skill…",4,"""id_3063"""


In [5]:
# outputs = json.load(open('../outputs.jsonl', "r"))
outputs = []
with open('../outputs_test.jsonl', "r") as f:
    lines = f.readlines()
    for line in lines:
        jsonline = json.loads(line)
        if not isinstance(jsonline, dict):
            continue
        outputs.append(jsonline)
df_outputs = pl.DataFrame(outputs)
# predictions = [o['prediction'] for o in outputs]

In [ ]:
threshold = 1
preds = (
    df_train.join(df_outputs, on="data_id", how="inner")
    .with_columns(
        (
            ((pl.col("match_score") - pl.col("score")).abs() <= threshold)
            .cast(pl.Int8)
            .alias("correct")
        )
    )["correct"]
    .to_list()
)
accuracy = sum(preds) / len(preds)
accuracy

0.4097693351424695

In [9]:
# df_pred = pd.DataFrame(data)
# (
#     df_train.iloc[indexes].reset_index(drop=True)
#     .join(df_pred)
# )